# **CMIG - Python Tutorials**

*The idea of this notebook is for you to read through the text and execute each cell as you go along, filling in commands/blocks of code as necessary. This is intended for people with little to no programming / python experience. If this is review for you, feel free to skim through it.*  

In [ ]:
################################################################################################
# 3/16/23 NOTE: THIS NEEDS TO BE EDITED TO BE COMPATIBLE WITH POSSIBLE NEW JHUB FILE STRUCTURE # 
################################################################################################

# Data Science: Xarray Fundamentals

OUTLINE:
- Xarray Intro
- Xarray data structures
- Selecting data
- Reductions
- Loading data from netCDF files
- Recap
- Exercises & Further Resources

---
<a id='section1'></a>
## Xarray Intro
---

In climate modeling (and the data sciences more widely), we often deal with multidimensional data. By multidimensional data (also often called N-dimensional), I mean data with many independent dimensions or axes. 

For example, we might represent Earth’s surface temperature as a four dimensional variable:

$$ T(x,y,z,t)$$

where 
$x$ is longitude, 
$y$ is latitude, 
$z$ is the vertical level, and 
$t$ is time.

We could even want to store multiple variables within the same dataset. 

Geo- and computer scientists have developed a python package called [Xarray](https://docs.xarray.dev/en/stable/getting-started-guide/why-xarray.html) that helps us keep track of the metadata and values associated with (usually gridded) high dimensional data. The point of xarray is to provide convenience for working with this type of data.
<div>
<img src=https://camo.githubusercontent.com/d19bddb6855355ed99d87d60975a8a85cc1807975f80a83d42912108e24e2a10/68747470733a2f2f676466612e7567722e65732f707974686f6e2f636c696d6174655f646174612f696d672f786172726179322e706e67 width="900">
<div>

Xarray essentially "wraps" metadata around NumPy arrays. In other words, the gridded values in an Xarray dataset are stored in NumPy arrays (which we've talked about before), but Xarray allows you to store and manipulate lots of helpful information such as dataset names, dimensions, coordinates, and other metadata, and also provides tons of useful built-in methods which make analysis very simple. 

<a id='section2'> </a>

---
## Xarray data structures
---

Xarray has two fundamental data structures:

- a `DataArray`, which holds a single multi-dimensional variable and its coordinates

- a `Dataset`, which holds multiple variables that potentially share the same coordinates

### DataArray

A DataArray has four essential attributes:

- `values`: a `numpy.ndarray` holding the array’s values. The actual numerical data of interest.

- `dims`: dimension names for each axis (e.g., ('x', 'y', 'z'))

- `coords`: a dict-like container of arrays (coordinates) that label each point (e.g., 1-dimensional arrays of numbers, datetime objects or strings)

- `attrs`: an OrderedDict to hold arbitrary metadata (attributes)

Let’s start by constructing some DataArrays manually:

In [ ]:
import os
import numpy as np
import xarray as xr # here we are importing xarray

# for plotting / visualizing:
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (8,5)

A simple DataArray without dimensions or coordinates isn’t much use:

In [ ]:
# the xr.DataArray() function is how we construct a DataArray
da = xr.DataArray([9, 0, 2, 1, 0])
da

*Note: Just like with displaying a variable's values, if you simply run a cell with your dataArray / dataset name you will be able to view its contents (variables, dims, coords, metadata, etc.)*

We could add a dimension name…

In [ ]:
da = xr.DataArray([9, 0, 2, 1, 0], dims=['year'])
da

But things get most interesting when we add a coordinate:

In [ ]:
da = xr.DataArray([9, 0, 2, 1, 0],
                  dims=['year'],
                  coords={'year': [2010, 2011, 2012, 2013, 2014]})
da

This coordinate has been used to create an *index*, in other words, a way to select certain data based on the values of that dimensions.

And, Xarray has built-in plotting using the `plot()` method.

In [ ]:
da.plot(marker='o')

### Dataset

An Xarray Dataset is simply a collection of multiple DataArrays that share some/all of their dims and coordinates.

Manually constructing datasets is a bit more complicated syntax-wise, but here is a brief example. 

For example:

In [ ]:
# here we are just putting together a made up dataset with var's temp and precip
# and one dimension (time)
# using made up data. Don't worry about the syntax for now!

tempdata = [58, 38, 49, 52, 57]
precipdata = [3, 0.1, 0.46, 1.1, 0.8]
timedata = [2010, 2011, 2012, 2013, 2014]

ds = xr.Dataset(data_vars=dict(temp=(['time'], tempdata),
                         precip=(['time'], precipdata)
                         ),
            coords=dict(time=timedata)
           )

ds

*You'll notice that the dataset has 2 entries under 'Data variables', and that the only dimension is time.*

You can use dictionary `key:value` syntax to access the values of data or coords in an xarray DataArray or Dataset. 

You can also use "attribute" syntax (e.g. `ds.temp`), but this may not always provide as much flexibility if you're trying to manipulate the data.

For example:

In [ ]:
# display the temp data from this ds
ds['temp']

# or: ds.temp

# Notice how ds['temp'] is type 'xarray.DataArray', that's because a dataset is just a collection of dataArrays.

### Multidimensional Dataset

Xarray’s real potential comes with multidimensional data.

Let's load data from a netCDF file into an xarray dataset. 

*(This would be the main way you'll be interacting with datasets. These netCDF (.nc) files, are one of the most common filetypes for storing gridded climate data and/or climate model output.)*

In [ ]:
# here we are using the xr.open_dataset() function to open some maximum global temperature data from 2018
ds_in = xr.open_dataset('/home/jovyan/shared/GEOG60/pythontutorials/tutorialdata/tasmax_2018.nc')
ds_in

**TRY IT:** Take a moment to click through the dataset and familiarize yourself with its dimensions, coords, and data variables. Can you figure out the spatial and temporal grain of the data (in other words, the size of a grid cell in lat and lon coordinates and the length of one 'timestep'?

### Coordinates vs. Data Variables

Data variables can be modified through arithmentic operations or other functions. **Coordinates are always keept the same.** This will be very helpful and important when we get to performing computations with Xarray data.

For example:

In [ ]:
ds_in * 1000

*Notice how the values on `mx2t` have changed, but nothing else has!*

Let's rename the 'mx2t' data variable to something a bit more descriptive.

In [ ]:
# NOTE: any changes you make to your dataArray/dataset (such as ds_in * 1000 above)
# will NOT be saved unless you use the assignment operator.

ds_in = ds_in.rename({'mx2t':'Tmax'}) # here we overwrite the old ds with a new one, renaming mx2t to Tmax.
ds_in

---
<a id='section3'></a>
# Selecting Data (Indexing)
---

We can always use regular numpy indexing and slicing on DataArrays:

In [ ]:
ds_in.Tmax[:, 5, 18] # this would be selecting all times, lat at index location 5, and lon at index loc 18

# note that the result only has the dimension 'time'

In [ ]:
# ...and here we are plotting the max temp for that lat, lon cell
ds_in.Tmax[:, 5, 18].plot()

In [ ]:
# alternatively:
ds_in.Tmax[4, :, :].plot()

However, it is often much more powerful to use xarray’s `.sel()` method to use ***label-based indexing.***

*This is part of what makes xarray so useful!*

In [ ]:
# Let's just select data for the date 7/31/18, and save that in a new var sel_731
sel_731 = ds_in.sel(time='2018-07-31')
sel_731

In [ ]:
sel_731.Tmax.plot()

*Note: as we showed in the practical, you don't even need to know the exact coordinates of the data you're selecting! If you wanted to just select the data closest to 7/25, you could write:*

```python
ds_in.sel(time='2018-07-25', method='nearest')
```

*Handy.*

`.sel()` also supports slicing. Unfortunately we have to use a somewhat awkward syntax, but it still works.

In [ ]:
ds_in

In [ ]:
# how many months worth of data have we selected here?
# try executing the code below. 

ds_in.sel(time=slice('2018-07-31', '2018-11-30'))

In [ ]:
# maybe we want to just look at the max temp in the northern hemisphere for july
n_hemis_toplot = ds_in.Tmax.sel(latitude=slice(90, 0), time='2018-07-31')
n_hemis_toplot.plot()

When slicing, the order of the values in your coordinates matters! What would happen if you tried to sel:
```python
n_hemis_toplot = ds_in.Tmax.sel(latitude=slice(0, 90), time='2018-07-31')
```

Any ideas why?

In [ ]:
# try it here:


<a id='section4'></a>

---
# Computation with Xarray objects
---

As previously mentioned, Xarray dataarrays and datasets work seamlessly with arithmetic operators and numpy array functions.

Say we wanted to convert our max temps (which are currently in Kelvin) to C.

In [ ]:
temp_C = ds_in.Tmax - 273.15
temp_C

# coords stay unchanged!

We can also combine multiple xarray datasets / dataArrays in arithemtic operations:

In [ ]:
# let's get difference in temp from july to december
summer_da = ds_in.Tmax.sel(time='2018-07-31')
winter_da = ds_in.Tmax.sel(time='2018-12-31')

diff_da = (summer_da - winter_da)
diff_da

In [ ]:
# surprise surprise, it was warmer in the July in the northern hemipshere, and
# vice-versa for the southern hemisphere
diff_da.plot()

This is particularly useful when you have various data variables. For instance, you may want to compute Q = P - ET by subtracting some ET dataset from some Precip dataset that you're working with.

<a id='section5'></a>

---
# Reductions
---

Just like in numpy, we can reduce xarray DataArrays along any number of axes.

Typically when we reduce something, we take the mean, min/max, sum, etc. across a given dimension. 

These are all very easy to do in xarray using the built-in methods `mean()`, `min()`, `max()`, etc. 

In [ ]:
# Let's look at the globe's average max temperature profile for July '18 as it varies by latitude:

# we selected data for july above in summer_da,
# use the '.mean()' function and specify which dimension we want to REDUCE i.e. take the mean across:
summer_da.mean(dim='longitude')

Notice how we're now only left with the latitude dimension after selecting for time and reducing along the lat dimension. Let's visualize this below:

In [ ]:
summer_da.mean(dim='longitude').plot()

# on average across the globe, max temps are higher in the northern hemisphere which makes sense since it's N. Hemisphere summer

What if we wanted to make a map of the max Tmax across the year for each grid point? Which dimension would we reduce across? Try to fill in the code below.

In [ ]:
# fill in the 'dims' argument(s) below!
max_tmax_da = ds_in.Tmax.max() # <-- FILL HERE

# verify by plotting
max_tmax_da.plot()

### Weighted Reductions

Sometimes we want to perform a reduction (e.g. a mean) where we assign different weight factors to each point in the array. Xarray supports this via [weighted array reductions](https://docs.xarray.dev/en/stable/user-guide/computation.html#weighted-array-reductions).



For example, you may want to weight by the cosine of latitude when taking a global mean to account for differences in grid cell size (lines of longitude get closer and closer the further away you get from the equator, meaning cells are all different sizes!)

In [ ]:
# make lat weights
latweights = np.cos(np.deg2rad(ds_in.latitude)) # taking the lat dim of the dataset, putting it in radians, taking the cosine
latweights.plot() # visualize lat weights

In [ ]:
# then take the weighted mean: 
tmax_weighted_globalmean = ds_in.Tmax.weighted(latweights).mean(dim=['latitude', 'longitude']) # note the .weighted() method where you supply your weights as an argument.
tmax_weighted_globalmean

In [ ]:
# plot, comparing to unweighted
tmax_unweighted_globalmean = ds_in.Tmax.mean(dim=['latitude', 'longitude']) # note the .weighted() method where you supply your weights as an argument.


# plot:
fig, ax = plt.subplots()

tmax_weighted_globalmean.plot(ax=ax, label='lat weights')
tmax_unweighted_globalmean.plot(ax=ax, label='unweighted')

ax.legend()

<a id='section6'></a>

---
# Loading Data from netCDF Files
---

NetCDF (Network Common Data Format) is the most widely used format for distributing geoscience data. NetCDF is maintained by the Unidata organization.

From the NetCDF website:

> NetCDF (network Common Data Form) is a set of interfaces for array-oriented data access and a freely distributed collection of data access libraries for C, Fortran, C++, Java, and other languages. The netCDF libraries support a machine-independent format for representing scientific data. Together, the interfaces, libraries, and format support the creation, access, and sharing of scientific data. NetCDF data is:
- Self-Describing. A netCDF file includes information about the data it contains.
- Portable. A netCDF file can be accessed by computers with different ways of storing integers, characters, and floating-point numbers.
- Scalable. A small subset of a large dataset may be accessed efficiently.
- Appendable. Data may be appended to a properly structured netCDF file without copying the dataset or redefining its structure.
- Sharable. One writer and multiple readers may simultaneously access the same netCDF file.
- Archivable. Access to all earlier forms of netCDF data will be supported by current and future versions of the software.

Xarray was designed to make reading netCDF files in python as easy, powerful, and flexible as possible. (See [xarray netCDF docs](https://docs.xarray.dev/en/latest/user-guide/io.html#netcdf) for more details.)

As mentioned above, the syntax is as follows:

```python
ds = xr.open_dataset('path_to_your_file.nc')
```

<a id='section7'></a>

---
# RECAP
---

## ...So, to recap:

Xarray is one of our best tools for viewing and manipulating gridded climate data.

Xarray has:
- `data` / variables: the numerical data representing whatever var/quantity you are interested in
- `dimensions`: names for each axis of the data -- ex. you could have some 3-d data with dims lat, lon, and time
- `coordinates`: the actual VALUES that correspond to the 'dimensions' above. So for lat, your coordinates could be [0, 0.5, 1, 1.5, 2, ...] which would correspond to those degrees of latitude in the data.
- `attributes`: metadata such as units, data description, version history, etc.

Data are organized in DataArrays or Datasets, which provide fancy labels over numpy arrays.

These labels allow for powerful indexing, slicing, computations, and reduction operations.

Data are typically stored in netCDF (.nc) file format. 

Xarray has tons of built in methods which make life easy!

<a id='section8'></a>

---
# Exercises
---

1. Open the file provided at the path below, saving it into the var `ds_2`.
2. What are the dimensions, coordinates, and data variables of the data? What climate model does this data come from?
3. What is the spatial and temporal grain/resolution?
4. Select Precip data for March 15, 2011.
5. Create a global mean precipitation timeseries from the data. **Challenge:** Weight your global mean by cosine of latitude.

In [ ]:
# 1. 
# path = '/home/jovyan/shared/GEOG60/pythontutorials/tutorialdata/pr_Amon_CESM2-WACCM-FV2_historical_r1i1p1f1_gn_200001-201412.nc'


In [ ]:
# 2. 


In [ ]:
# 3. 


In [ ]:
# 4


In [ ]:
# 5


---
# Further Resources
---

There are tons of other things you can do with Xarray. When in doubt, google it or look on the Xarray Documentation!

- [Xarray Docs](https://docs.xarray.dev/en/latest/index.html)
- [Xarray Fundamentals (tutorial)](https://earth-env-data-science.github.io/lectures/xarray/xarray.html#xarray-fundamentals)
- [Xarray Tutorial - Pangeo Gallery](https://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/xarray.html)